<a href="https://colab.research.google.com/github/adrenalinovaya/RL/blob/main/Vanilla_DQN_%D0%94%D0%97_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LunarLander with DQN algorithm + TargetQ + ReplayBuffer

Этот пример демонстрирует применение алгоритма DQN для задачи обучения агента на среде CartPole-v1 из пакета Gymnasium на базе PyTorch.

Original link: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#dqn-algorithm

## Imports

In [1]:
# @title установка драйверов
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install -y swig build-essential python-dev python3-dev > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!apt-get install xvfb > /dev/null 2>&1

In [2]:
# @title установка библиотек
!pip install rarfile --quiet
!pip install stable-baselines3[extra] --quiet
!pip install ale-py --quiet
!pip install gym[box2d] --quiet
!pip install pyvirtualdisplay --quiet
!pip install pyglet --quiet
!pip install pygame --quiet
!pip install minigrid --quiet
!pip install -q swig --quiet
!pip install -q gymnasium[box2d] --quiet
!pip install 'minigrid<=2.1.1' --quiet
!pip3 install box2d-py --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 24.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building whee

In [4]:
# Imports
import io
import os
import glob
import torch
import base64

import numpy as np
import matplotlib.pyplot as plt

import sys
import gymnasium
sys.modules["gym"] = gymnasium

import stable_baselines3
from stable_baselines3 import DQN
from stable_baselines3.common.results_plotter import ts2xy, load_results
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_atari_env

import gymnasium as gym
from gym import spaces
from gym.envs.box2d.lunar_lander import *
from gym.wrappers.monitoring.video_recorder import VideoRecorder
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from gym.envs.box2d.lunar_lander import *
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from gym.envs.box2d.lunar_lander import *
import stable_baselines3
from stable_baselines3 import DQN
from stable_baselines3.common.results_plotter import ts2xy, load_results
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_atari_env

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import warnings
warnings.simplefilter("ignore")

In [6]:
torch.__version__

'2.4.0+cu121'

In [7]:
gym.__version__

'0.29.1'

In [8]:
device

device(type='cuda')

Когда агент наблюдает за текущим состоянием среды и выбирает действие, среда переходит в новое состояние, а также возвращает вознаграждение, указывающее на последствия этого действия. В данной задаче награда равна +1 за каждый инкрементный временной шаг, и среда завершается, если шест падает слишком далеко или тележка удаляется от центра более чем на 2,4 единицы. Это означает, что сценарии с лучшими результатами будут выполняться дольше, накапливая большую отдачу.

Задача CartPole построена таким образом, что входными данными для агента являются 4 реальных значения, представляющие состояние среды (положение, скорость и т.д.). Мы берем эти 4 входных сигнала без какого-либо масштабирования и пропускаем их через небольшую полносвязную сеть с 2 выходами, по одному на каждое действие. Сеть обучается предсказывать ожидаемое значение для каждого действия с учетом входного состояния. Затем выбирается действие с наибольшим ожидаемым значением.

### Replay Memory

Для обучения DQN мы будем использовать Replay Memory. В ней хранятся переходы, которые наблюдал агент, что позволяет нам использовать эти данные в дальнейшем. Переходы извлекаются путем случайной выборки, что позволяет избежать корреляции между наблюдениями. Это значительно стабилизирует и улучшает процедуру обучения DQN.

Для этого нам понадобятся два класса:

`Transition` - именованный кортеж, представляющий один переход в нашем окружении. По сути, он сопоставляет пары (состояние, действие) с их результатом (следующее_состояние, награда), причем состояние - это изображение разницы на экране.

`ReplayMemory` - циклический буфер ограниченного размера, в котором хранятся переходы. В нем также реализован метод `.sample()` для выбора случайной выборки переходов для обучения.

In [9]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [10]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Наша модель будет представлять собой нейронную сеть, которая учитывает разницу между текущим и предыдущим экранами. Она имеет два выхода, представляющих собой $Q(s,left)$ и $Q(s,right)$ (где $s$ - вход сети). По сути, сеть пытается предсказать ожидаемую прибыль от каждого действия с учетом текущих входных данных.

In [23]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 256)
        self.layer2 = nn.Linear(256, 256)
        self.layer3 = nn.Linear(256, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

**Отображение состояния среды, сохранение записи эпизода, просмотр записи эпизода**

In [12]:
def display_state(state):
    plt.figure(figsize=(8, 6))
    plt.imshow(state)
    plt.axis('off')


def record_episode(eps_frames, records_dir, agent_name, exp_id, eps_num):
    record_path = os.path.join(records_dir, f'{agent_name}_{exp_id}_eps-{eps_num}.mp4')
    eps_frame_dir = 'episode_frames'
    os.mkdir(eps_frame_dir)

    for i, frame  in enumerate(eps_frames):
        PIL.Image.fromarray(frame).save(os.path.join(eps_frame_dir, f'frame-{i+1}.png'))

    os.system(f'ffmpeg -r 16 -i {eps_frame_dir}/frame-%1d.png -vcodec libx264 -b 10M -y "{record_path}"');
    sh.rmtree(eps_frame_dir)


def show_episode_records(records_dir):
    record_paths = glob(os.path.join(records_dir, "*.mp4"))
    html_str = ''
    for i, record_path in enumerate(record_paths):
        mp4 = open(record_path, 'rb').read()
        data = f"data:video/mp4;base64,{b64encode(mp4).decode()}"
        html_str += f'EPISODE # {i+1}<br><video width=500 controls><source src="{data}" type="video/mp4"></video><br><br>'
    return HTML(html_str)


**Создание директорий для логирования результатов, сохрание параметров и метрик экспериментов**

In [13]:
def create_exp_dirs(env_dir, exp_params):
    dirs = dict()
    dirs['exp'] = os.path.join(env_dir, 'experiments', exp_params["algorithm_name"], exp_params["exp_id"])
    dirs['training'] = os.path.join(dirs['exp'], 'training')
    dirs['evaluation'] = os.path.join(dirs['exp'], 'evaluation')
    os.makedirs(dirs['training'], exist_ok=True)
    os.makedirs(dirs['evaluation'], exist_ok=True)
    return dirs


def save_exp_params(params, exp_dir):
    params_path = os.path.join(exp_dir, 'experiment_params.json')
    with open(params_path, 'w') as f:
        json.dump(params, f)


def save_metrics(metrics, metrics_type, train_eps_dir):
    metrics_path = os.path.join(train_eps_dir, f'{metrics_type}_metrics.json')
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f)


## Training

### Hyperparameters and utilities

Далее мы инициализируем модель и ее оптимизатор, а также определяем некоторые утилиты:

`select_action` - выбор действия в соответствии с эпсилон-жадной политикой. Проще говоря, иногда мы будем использовать нашу модель для выбора действия, а иногда - просто равномерную выборку. Вероятность выбора случайного действия будет начинаться с `EPS_START` и экспоненциально убывать по направлению к `EPS_END`. `EPS_DECAY` управляет скоростью затухания.

`plot_durations` - помощник для построения графика продолжительности эпизодов, а также среднего значения за последние 100 эпизодов (показатель, используемый в оценках). График будет находиться под ячейкой, содержащей основной цикл обучения, и обновляться после каждого эпизода. По умолчанию продолжительность эпизода в CartPole 500 действий. Если обученный агент сможет удержать шест вертикально на протяжении 500 эпизодов - можно считать, что он хорошо обучился.

In [14]:
env_name = 'LunarLander-v2'
env = gym.make(env_name)
print('Пространство наблюдений (состояний): ', env.observation_space.shape)
print('Пространство действий:: ', env.action_space.n)

Пространство наблюдений (состояний):  (8,)
Пространство действий::  4


In [25]:
# BATCH_SIZE - количество эпизодов, отобранных из буфера воспроизведения
# GAMMA - коэффициент дисконтирования
# EPS_START - начальное значение эпсилон
# EPS_END - конечное значение эпсилон
# EPS_DECAY - скорость экспоненциального спада эпсилон, чем больше - тем медленнее падение
# TAU - скорость обновления целевой сети
# LR - скорость обучения оптимизатора ``AdamW``.

BATCH_SIZE = 256
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.01 #0.005
LR = 1e-4
FULL_MEMORY_LENGTH = 10000

# Среда
env = gym.make('LunarLander-v2')



# Получить число действий
n_actions = env.action_space.n
# Получить число степеней свободы состояний
state, info = env.reset()
n_observations = len(state)

# Инициилизировать сети: целевую и политики
policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)

# Подгрузить в целевую сеть коэффициенты из сети политики
target_net.load_state_dict(policy_net.state_dict())

# Задать оптимайзер
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)

# Инициализировать Replay Memory buffer
memory = ReplayMemory(FULL_MEMORY_LENGTH)


steps_done = 0

#     массив длительности эпизода - пойдет в отчет о том, сколько продержался агент
episode_durations = []

In [26]:
# еpsilon-жадная стратегия выбора действия
def select_action(state):
    global steps_done  # глобальный шаги для убывания epsilon
#     случайное значение для определения какой шаг будем делать жадный или случайный
    sample = random.random()

#     установка порога принятия решения - уровня epsilon
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)

#     увеличиваем счетчик шагов
    steps_done += 1

#     если случайный порог больше epsilon-порога
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) вернет наибольшее значение столбца в каждой строке.
            # Второй столбец в результате max - это индекс того места,
            # где был найден максимальный элемент,
            # поэтому мы выбираем действие с наибольшим ожидаемым вознаграждением.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        # Иначы выбираем случайное дайствие
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

## Training loop

Обучение модели.

Функция `optimize_model` выполняет один шаг оптимизации. Сначала она производит выборку, конкатенирует все тензоры в один, вычисляет
$Q(s_t, a_t)$ и $V(s_{t+1}) = max_aQ(s_{t+1},a)$ и объединяет их в Loss. По определению мы задаем $V(s)=0$, если $s$ является терминальным состоянием. Для повышения устойчивости мы также используем целевую сеть для вычисления $V(s_{t+1})$. Целевая сеть обновляется на каждом шаге с помощью __мягкого обновления__, управляемого гиперпараметром `TAU`.

### Мягкое обновление целевой сети



In [27]:
def optimize_model(batch_size: int):
    if len(memory) < batch_size:
        return

    # Получить из памяти батч
    transitions = memory.sample(batch_size)
    # Преобразовать его в namedtuple
    batch = Transition(*zip(*transitions))

    # Вычислить маску нефинальных состояний и соединить элементы батча
    # (финальным состоянием должно быть то, после которого моделирование закончилось)

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                        batch.next_state)), device=device, dtype=torch.bool)

    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])

    # Собираем батчи для состояний, действий и наград
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Вычислить Q(s_t, a) - модель вычисляет Q(s_t),
    # затем мы выбираем столбцы предпринятых действий.
    # Это те действия, которые были бы предприняты для каждого состояния партии в соответствии с policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Вычислить V(s_{t+1}) для всех следующих состояний.
    # Ожидаемые значения действий для не_финальных_следующих_состояний вычисляются
    # на основе "старшей" целевой_сети; выбирается их наилучшее вознаграждение с помощью max(1)[0].
    # Это объединяется по маске, так что мы будем иметь либо ожидаемое значение состояния,
    # либо 0, если состояние было финальным.
    next_state_values = torch.zeros(batch_size, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Вычисляем ожидаемые Q значения
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Объединяем все в общий лосс
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Готовим градиент
    optimizer.zero_grad()
    loss.backward()
    # Обрезаем значения градиента - проблемма исчезающего/взрывающего градиента
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)

    optimizer.step()


Ниже представлен основной цикл обучения.

В начале мы сбрасываем окружение и получаем тензор начального `state`. Затем мы выбираем действие, выполняем его, наблюдаем следующее состояние и вознаграждение (всегда 1) и один раз оптимизируем нашу модель. Когда эпизод заканчивается (модель не работает), мы перезапускаем цикл.


`num_episodes` устанавливается равным 1500, если доступен графический процессор, в противном случае планируется 500 эпизодов, чтобы обучение не занимало слишком много времени.

Обучение RL-агентов может быть случайным процессом, поэтому перезапуск обучения может дать лучшие результаты, если сходимость не наблюдается.

In [29]:
if torch.cuda.is_available():
    num_episodes = 2000 #1500
else:
    num_episodes = 500
total_reward = []

for i_episode in tqdm(range(num_episodes)):
    episode_reward = 0
    # Для каждого эпизода инициализируем начальное состояние
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

    # выполняем действия пока не получим флаг done
    # t - считает сколько шагов успели сделать пока шест не упал
    for t in count():
        # выбираем действие [0, 1]
        action = select_action(state)
        # Делаем шаг
        observation, reward, terminated, truncated, _ = env.step(action.item())
        episode_reward += reward

        # Преобразуем в тензор
        reward = torch.tensor([reward], device=device)

        # Объединяем done по двум конечным состояниям
        done = terminated or truncated

        # присваиваем следующее состояние
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # отправляем в память
        memory.push(state, action, next_state, reward)

        # переходим на следующее состояние
        state = next_state

        # запускаем обучение сети
        optimize_model(batch_size=128)

        # делаем "мягкое" обновление весов
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()

        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        # Если получили terminated or truncated завершаем эпизод обучения
        if done:
            # добавляем в массив продолжительность эпизода
            episode_durations.append(t + 1)
            total_reward.append(episode_reward)
            break

print('Complete')


100%|██████████| 2000/2000 [38:13<00:00,  1.15s/it]

Complete


In [30]:
# Вычисление скользящего среднего
window_size = 50
moving_avg = np.convolve(total_reward, np.ones(window_size)/window_size, mode='valid')
plt.plot(moving_avg)
plt.title('DQN + ReplayBuffer')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()